<a href="https://colab.research.google.com/github/SaeidDaliriSusefi/Grace_Downsacling/blob/main/Grace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import ee
import geemap

In [3]:
ee.Authenticate()
ee.Initialize(project="Your google earth engine project name", opt_url='https://earthengine-highvolume.googleapis.com')

In [100]:
map=geemap.Map(basemap="SATELLITE")
map

In [68]:
roi = map.draw_last_feature.geometry()

In [21]:
time_start = '2009'
time_end   = '2010'

In [30]:
def convert_to_monthly_sum(collection, start, end):
    start_date = ee.Date(start)
    end_date = ee.Date(end)

    adjusted_end = ee.Algorithms.If(
        ee.Number(end_date.get('day')).eq(1),
        end_date.advance(-1, 'day'),
        end_date
    )
    end_date = ee.Date(adjusted_end)

    months = ee.List.sequence(0, end_date.difference(start_date, 'month'))

    def map_month(m):
        m = ee.Number(m)
        start_m = start_date.advance(m, 'month')
        end_m = start_m.advance(1, 'month')

        filtered = collection.filterDate(start_m, end_m)
        monthly_image = filtered.sum() \
            .set('system:time_start', start_m.millis())

        first_image = ee.Image(filtered.first())
        monthly_image = monthly_image.copyProperties(first_image, first_image.propertyNames())

        return monthly_image

    return ee.ImageCollection.fromImages(months.map(map_month))


def convert_to_monthly_average(collection, start, end):
    start_date = ee.Date(start)
    end_date = ee.Date(end)

    adjusted_end = ee.Algorithms.If(
        ee.Number(end_date.get('day')).eq(1),
        end_date.advance(-1, 'day'),
        end_date
    )
    end_date = ee.Date(adjusted_end)

    months = ee.List.sequence(0, end_date.difference(start_date, 'month'))

    def map_month(m):
        m = ee.Number(m)
        start_m = start_date.advance(m, 'month')
        end_m = start_m.advance(1, 'month')

        filtered = collection.filterDate(start_m, end_m)
        monthly_image = filtered.mean() \
            .set('system:time_start', start_m.millis())

        first_image = ee.Image(filtered.first())
        monthly_image = monthly_image.copyProperties(first_image, first_image.propertyNames())

        return monthly_image

    return ee.ImageCollection.fromImages(months.map(map_month))


In [100]:
grace_55km = (
    ee.ImageCollection("NASA/GRACE/MASS_GRIDS_V04/MASCON_CRI")
    .select('lwe_thickness')
    .filterDate(time_start, time_end)
    .map(lambda img: img.toInt()
         .copyProperties(img, img.propertyNames())
         .set('date', img.date().format('YYYY-MM')))
)

grace_55km


In [100]:
precipitation = (
    ee.ImageCollection("NASA/GPM_L3/IMERG_MONTHLY_V06")
    .select("precipitation")
    .filterDate(time_start, time_end)
    .map(lambda img: img.toInt()
         .copyProperties(img, img.propertyNames())
         .set('date', img.date().format('YYYY-MM')))
)


precipitation_monthly = convert_to_monthly_sum(precipitation, time_start, time_end)

precipitation_monthly


In [100]:
air_temperature = (
    ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY_AGGR")
    .select("temperature_2m")
    .filterDate(time_start, time_end)
    .map(lambda img: img.toInt()
         .copyProperties(img, img.propertyNames())
         .set('date', img.date().format('YYYY-MM')))
)

air_temperature_monthly = convert_to_monthly_average(air_temperature, time_start, time_end)
air_temperature_monthly

In [100]:
land_temperature_monthly = (
    ee.ImageCollection("Oxford/MAP/LST_Day_5km_Monthly")
    .select('Mean')
    .filterDate(time_start, time_end)
    .map(lambda img: img.toInt()
         .copyProperties(img, img.propertyNames())
         .set('date', img.date().format('YYYY-MM')))
)

land_temperature_monthly


In [100]:

humidity = (
    ee.ImageCollection("UCSB-CHG/CHIRTS/DAILY")
    .select("relative_humidity")
    .filterDate(time_start, time_end)
    .map(lambda img: img.toInt()
         .copyProperties(img, img.propertyNames())
         .set('date', img.date().format('YYYY-MM')))
)


humidity_monthly = convert_to_monthly_average(humidity, time_start, time_end)
humidity_monthly


In [100]:
evaporation = (
    ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR")
    .select('potential_evaporation_sum')
    .filterDate(time_start, time_end)
    .map(lambda img: img.toInt()
         .copyProperties(img, img.propertyNames())
         .set('date', img.date().format('YYYY-MM')))
)


evaporation_monthly = convert_to_monthly_average(evaporation, time_start, time_end)
evaporation_monthly


In [100]:
runoff = (
    ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR")
    .select('runoff_sum')
    .filterDate(time_start, time_end)
    .map(lambda img: img.toInt()
         .copyProperties(img, img.propertyNames())
         .set('date', img.date().format('YYYY-MM')))
)


runoff_monthly = convert_to_monthly_sum(runoff, time_start, time_end)
runoff_monthly

In [100]:
ndvi = (
    ee.ImageCollection("MODIS/061/MOD13A2")
    .select('NDVI')
    .filterDate(time_start, time_end)
    .map(lambda img: img.toInt()
         .copyProperties(img, img.propertyNames())
         .set('date', img.date().format('YYYY-MM')))
)


ndvi_monthly = convert_to_monthly_average(ndvi, time_start, time_end)
ndvi_monthly

In [100]:
def join_collections(primary, secondary, band_name):
    join = ee.Join.inner()
    filter = ee.Filter.equals(leftField='date', rightField='date')
    joined = join.apply(primary, secondary, filter)

    return ee.ImageCollection(joined.map(lambda feature:
        ee.Image(feature.get('primary')).addBands(
            ee.Image(feature.get('secondary')).rename(band_name)
        )
    ))


joined1 = join_collections(grace_55km, precipitation_monthly, 'Precipitation')
joined2 = join_collections(joined1, air_temperature_monthly, 'Air temperature')
joined3 = join_collections(joined2, land_temperature_monthly, 'Land Surface temperature')
joined4 = join_collections(joined3, humidity_monthly, 'Humidity')
joined5 = join_collections(joined4, evaporation_monthly, 'Evaporation')
joined6 = join_collections(joined5, runoff_monthly, 'Runoff')
data_collection = join_collections(joined6, ndvi_monthly, 'NDVI')

data_collection


In [100]:
def classify_grace10km(img):
    training = img.stratifiedSample(
        numPoints=100,
        classBand='lwe_thickness',
        region=roi,
        scale=10000
    )

    model = ee.Classifier.smileRandomForest(80).train(
        features=training,
        classProperty='lwe_thickness',
        inputProperties=img.bandNames()
    ).setOutputMode('REGRESSION')

    bands = img.bandNames().remove('lwe_thickness')

    result = img.select(bands).classify(model).rename('grace10km').toFloat()

    return result.copyProperties(img, img.propertyNames())

grace10km = data_collection.map(classify_grace10km)

grace10km


In [100]:
count = grace10km.size().getInfo()
image_list = grace10km.toList(count)

Map = geemap.Map(basemap="SATELLITE")

for i in range(count):
    img = ee.Image(image_list.get(i)).clip(roi)

    date = img.get('date').getInfo()

    vmin = img.reduceRegion(
        reducer=ee.Reducer.min(),
        geometry=roi,
        bestEffort=True
    ).get('grace10km').getInfo()
    vmax = img.reduceRegion(
        reducer=ee.Reducer.max(),
        geometry=roi,
        bestEffort=True
    ).get('grace10km').getInfo()

    vis = {
        'min': vmin,
        'max': vmax,
        'palette': ['black', 'white']
    }

    Map.addLayer(img, vis, date)

Map.centerObject(roi, 6)
Map